## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Sayyan,YE,15.1718,44.3244,62.71,67,81,3.60,broken clouds,2021-07-25 03:06:58
1,1,Kahului,US,20.8947,-156.4700,85.26,83,40,24.16,scattered clouds,2021-07-25 03:06:59
2,2,Rikitea,PF,-23.1203,-134.9692,69.49,56,0,0.18,clear sky,2021-07-25 03:06:59
3,3,Qaanaaq,GL,77.4840,-69.3632,38.30,67,100,4.50,overcast clouds,2021-07-25 03:06:59
4,4,Sorland,NO,67.6670,12.6934,52.65,94,100,13.22,overcast clouds,2021-07-25 03:07:00


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
1,1,Kahului,US,20.8947,-156.4700,85.26,83,40,24.16,scattered clouds,2021-07-25 03:06:59
10,10,Bogande,BF,12.9714,-0.1436,77.95,73,11,7.63,few clouds,2021-07-25 03:07:01
11,11,Nishihara,JP,26.1842,127.7558,87.46,88,75,19.57,broken clouds,2021-07-25 03:07:02
16,16,Dingle,PH,10.9995,122.6711,86.52,68,82,18.48,broken clouds,2021-07-25 03:07:03
17,17,Aripuana,BR,-9.1667,-60.6333,75.07,51,35,1.41,scattered clouds,2021-07-25 03:05:09
22,22,Bandarbeyla,SO,9.4942,50.8122,80.56,55,35,23.55,scattered clouds,2021-07-25 03:07:04
25,25,Natchez,US,31.5604,-91.4032,82.62,89,1,0.00,clear sky,2021-07-25 03:07:05
26,26,Karratha,AU,-20.7377,116.8463,78.66,32,69,11.32,broken clouds,2021-07-25 03:07:05
27,27,Atuona,PF,-9.8000,-139.0333,78.42,70,15,17.85,few clouds,2021-07-25 03:07:06
32,32,Algiers,DZ,36.7525,3.0420,78.62,61,40,3.44,scattered clouds,2021-07-25 03:05:58


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                257
City                   257
Country                257
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
Date                   257
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities = preferred_cities_df.dropna(how="any")
clean_preferred_cities.count()

City_ID                257
City                   257
Country                257
Lat                    257
Lng                    257
Max Temp               257
Humidity               257
Cloudiness             257
Wind Speed             257
Current Description    257
Date                   257
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,85.26,scattered clouds,20.8947,-156.4700,
10,Bogande,BF,77.95,few clouds,12.9714,-0.1436,
11,Nishihara,JP,87.46,broken clouds,26.1842,127.7558,
16,Dingle,PH,86.52,broken clouds,10.9995,122.6711,
17,Aripuana,BR,75.07,scattered clouds,-9.1667,-60.6333,
22,Bandarbeyla,SO,80.56,scattered clouds,9.4942,50.8122,
25,Natchez,US,82.62,clear sky,31.5604,-91.4032,
26,Karratha,AU,78.66,broken clouds,-20.7377,116.8463,
27,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,
32,Algiers,DZ,78.62,scattered clouds,36.7525,3.0420,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,85.26,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Bogande,BF,77.95,few clouds,12.9714,-0.1436,Auberge Wendkouni
11,Nishihara,JP,87.46,broken clouds,26.1842,127.7558,Minshuku Agaihama
16,Dingle,PH,86.52,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
17,Aripuana,BR,75.07,scattered clouds,-9.1667,-60.6333,
22,Bandarbeyla,SO,80.56,scattered clouds,9.4942,50.8122,JABIR HOTEL
25,Natchez,US,82.62,clear sky,31.5604,-91.4032,Hampton Inn & Suites Natchez
26,Karratha,AU,78.66,broken clouds,-20.7377,116.8463,ibis Styles Karratha
27,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,Villa Enata
32,Algiers,DZ,78.62,scattered clouds,36.7525,3.0420,Sofitel Algiers Hamma Garden


In [10]:
hotel_df.count()

City                   257
Country                257
Max Temp               257
Current Description    257
Lat                    257
Lng                    257
Hotel Name             257
dtype: int64

In [11]:
#Identify what data type Hotel Name is
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [12]:
#Convert Hotel Name to NaN
inter_hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)
inter_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,85.26,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Bogande,BF,77.95,few clouds,12.9714,-0.1436,Auberge Wendkouni
11,Nishihara,JP,87.46,broken clouds,26.1842,127.7558,Minshuku Agaihama
16,Dingle,PH,86.52,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
17,Aripuana,BR,75.07,scattered clouds,-9.1667,-60.6333,NaN
22,Bandarbeyla,SO,80.56,scattered clouds,9.4942,50.8122,JABIR HOTEL
25,Natchez,US,82.62,clear sky,31.5604,-91.4032,Hampton Inn & Suites Natchez
26,Karratha,AU,78.66,broken clouds,-20.7377,116.8463,ibis Styles Karratha
27,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,Villa Enata
32,Algiers,DZ,78.62,scattered clouds,36.7525,3.0420,Sofitel Algiers Hamma Garden


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = inter_hotel_df.dropna()
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kahului,US,85.26,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
10,Bogande,BF,77.95,few clouds,12.9714,-0.1436,Auberge Wendkouni
11,Nishihara,JP,87.46,broken clouds,26.1842,127.7558,Minshuku Agaihama
16,Dingle,PH,86.52,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
22,Bandarbeyla,SO,80.56,scattered clouds,9.4942,50.8122,JABIR HOTEL
25,Natchez,US,82.62,clear sky,31.5604,-91.4032,Hampton Inn & Suites Natchez
26,Karratha,AU,78.66,broken clouds,-20.7377,116.8463,ibis Styles Karratha
27,Atuona,PF,78.42,few clouds,-9.8000,-139.0333,Villa Enata
32,Algiers,DZ,78.62,scattered clouds,36.7525,3.0420,Sofitel Algiers Hamma Garden
34,Umm Lajj,SA,83.79,clear sky,25.0213,37.2685,Moon light Furnished Units


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))